<B> sklearn을 이용해서 결측치 채우기 </B> 
- sklearn에는 결측값을 변환가능 : SimpleImputer, IterativeImputer, KNNImputer  
1) from sklearn.impute import SimpleImputer  
 : Mean(숫자형), Median(숫자형), Mode(문자형)로 결측치를 대체할 수 있음   
2) IterativeImputer : 예측기반 결측치 대체     
3) KNNImputer : 누락 된 값이있는 항목과 항목 사이의 "거리"를 비교해서 결측치를 대체함   

### 1. 결측값 
- 결측값을 버리고 관측값만을 가지고 분석하게 되면 일반적으로 통계적인 편향이 발생하게 된다. 따라서 결측값에 대체값얼 넣어 분석하게 되는데 이를 결측값대체(Imputation)라고 한다. 

### 1) Imputation
#### (1) SimpleImputer
- 기본 통계치(=대표값)를 사용하여 imputation 하는 방법임
- 대표값을 사용하는 방법이기 때문에 빠르다는 장점이 있으나, 단점으로는   
   1) 다른 피처간의 상관도를 전혀 고려하지 않는다.    
   2) 비슷한 느낌으로 경향성에 대한 고려가 없다.   
   3) 정확도가 떨어진다   
   4) 평균의 경우 분산이 줄어든다   
   5) 최빈값의 경우 데이터 전체에 편향이 생긴다  
       
       
- 사용방법   

<U> 각 열에서의 최빈값으로 null을 대체 </U> 

> from sklearn.impute import SimpleImputer

<U> strategy : </U>  
<I> 'mean': 평균값 (디폴트)  
    'median': 중앙값  
    'most_frequent': 최빈값  
    'constant': 특정값, 예 SimpleImputer(strategy='constant', fill_value=1)  </I>

> SimpleImputer(missing_values=np.nan, strategy='most_frequent')

#### (2) Iterativeimputer
- 결측치에 대한 대치 방법으로 선형 회귀 원리를 이용하는 방법임. 
- 예측하고자 하는 y변수에 대해 여러 input 변수들을 활용하여 y를 예측할 수 있는 함수에 맞춰 np.nan을 예측함. 
- 결측 값이 다양한 변수에서 발생하는 경우, 사용 가능 변수 수가 적어 적합한 모델을 만들기 어렵고, 또 이렇게 만들어진 모델의 예측력은 낮으므로 사용하기 어려움 


- 사용방법  
  
<U> 사이킷런에서는 Impute를 하기 위해서는 두가지 매서드를 불러와야 함</U> 

> from sklearn.experimental import enable_iterative_imputer  
> from sklearn.impute import IterativeImputer

<U> verbose = False : 실행 결과를 반복 출력하는 것을 막기위한 설정 </U>  
<U> 출력 결과는 Array 형태이므로, dataframe으로 변경 필요 </U>
    
> pd.DataFrame(IterativeImputer(verbose=False).fit_transform(total))

#### (3) KNNImputer 
- KNN이란? : k-최근접 이웃 알고리즘으로 분류나 회귀에 사용되는 비모수 방식으로, 임의의 k 포인트에 대해 가장 근접한 최근접 이웃을 찾는 알고리즘임 
- KNN 알고리즘의 특성을 이용하여 결측치를 찾는데 활용되는 방법임
- 원하는 인접 이웃 수의 "거리"에 대한 가중 또는 가중 평균을 사용하여 결측값을 대치함 
- 단순 평균을 사용하는 것보다는 효과적이고 정확한 방법이지만, 대규모 데이터 세트에서는 런타임이 많이 증가하게 됨, 또한 feature수가 너무 많을 경우 차원 문제의 저주에 걸릴 수도 있음 

- 사용방법 : 
>  imputer = KNNimputer(n_neighbors=2)   
>  imputer.fit_transform(data)

#### (4) 그외 

- df.fillna() 방법
  > df.fillna(0)   
  > df.fillna(method='pad') : 이전 값으로 채워줌   
  > df.fillna(method='bfill') : 뒤의 값으로 채워줌   
  > df.replace(to_replace=np.nan, value= -3) 

## 2. OOB

### 1) out-of-bag score 
- 랜덤포레스트 모델을 사용할 경우 성능평가를 하는 방법 중 하나로, 예측이 얼마나 정확한가에 대한 추정을 수치로 나타낸 것
- 사실 상 자세히 살펴보면 test데이터셋에서 성능평가를 하는 방식과 동일함. 
- 트리들을 훈련시킨 후에, 부스트랩핑한 데이터들과 원본데이터를 비교해 특정 관측치를 포함하지 않는 부트스트랩 데이터를 (약 36.8%)을 따로 모으는데, 이떄의 데이터를 OOB예제들이라고 불림 
- 이 예제들을 따로 랜덤포레스트에 학습을 시켜 결과적으로 나온 Vote들의 합이 OOB error임 
- OOB 측정을 통해 따로 test셋을 구성할 필요가 없어짐 

#### 2) 사용방법 
> RandomForestRegressor(oob_score=True)